# Ingest race results data

In [0]:
%run ../SetUp

In [0]:
json_file_path = session_helper.get_storage_account_url(folder="raw", file="results.json")

results_df = (
    spark.read
    .format("json")
    .option("inferSchema", True)
    .load(json_file_path)
)

#results_df.display()

In [0]:
import pyspark.sql.functions as F

transformed_results_df = (
    results_df
    .withColumnRenamed("raceId", "race_id")
    .withColumnRenamed("resultId", "result_id")
    .withColumnRenamed("driverId", "driver_id")
    .withColumnRenamed("constructorId", "constructor_id")
    .withColumnRenamed("fastestLap", "fastest_lap")
    .withColumnRenamed("fastestLapSpeed", "fastest_lap_speed")
    .withColumnRenamed("fastestLapTime", "fastest_lap_time")
    .withColumnRenamed("positionOrder", "position_order")
    .withColumnRenamed("positionText", "position_text")
    .withColumn("ingestion_date", F.current_timestamp())
    .drop("statusId")
)

#transformed_results_df.display()

In [0]:
spark.sql("DROP TABLE IF EXISTS dev.results_bronze")
transformed_results_df.write.format("delta").saveAsTable("dev.results_bronze", mode="overwrite")